In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv
/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv
/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv
/kaggle/input/ue20cs344-nam-assignment-2/features.pt


In [54]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade torch-cluster
!pip install gensim

^C
ERROR: Operation cancelled by user
  Using cached torch_cluster-1.6.1.tar.gz (53 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.1-cp37-cp37m-linux_x86_64.whl size=739190 sha256=a477cdfabf8cf7ec2b016785c894cf2e1ba17d2fcd32cee997b5f5e589296655
  Stored in directory: /root/.cache/pip/wheels/59/a4/3a/bd84e20d689ff3f2478a7bb718c3a5855710d5c77a054df5fb
Successfully built torch-cluster
/bin/bash: pip install gensim: command not found


In [57]:
pip install -U gensim

Note: you may need to restart the kernel to use updated packages.


In [58]:

!python -m pip install -U gensim

In [59]:
!pip install torch-geometric

In [60]:
import networkx as nx 
import torch
import pandas as pd
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn as nn

In [61]:
df_nodes=pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv")
df_edges=pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv")

In [62]:
df_nodes.head()

,id,branch,train_mask,test_mask
0,0,ECE,True,False
1,1,ECE,True,False
2,2,CSE,True,False
3,3,EEE,True,False
4,4,NaN,False,True


In [63]:
from sklearn.preprocessing import LabelEncoder

ENCODING CATEGORICAL TO NUMERICAL

In [64]:
labels = LabelEncoder()
df_nodes['branch']=labels.fit_transform(df_nodes['branch'])

In [65]:
df_nodes.head()

,id,branch,train_mask,test_mask
0,0,1,True,False
1,1,1,True,False
2,2,0,True,False
3,3,2,True,False
4,4,3,False,True


Trained without embeddings and used the file given

In [66]:
features =torch.load("/kaggle/input/ue20cs344-nam-assignment-2/features.pt")

In [67]:
G= nx.from_pandas_edgelist(df_edges, 'source','target')

In [68]:
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x=features
y=torch.tensor(df_nodes['branch'].values,dtype=torch.long)
data = Data(x=x,edge_index=edge_index, y=y)
data.train_mask =df_nodes['train_mask'].values
data.test_mask = df_nodes['test_mask'].values

In [69]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)




In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_features,16,3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [71]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(200):
    loss = train()
    print('Epoch: {}, Loss: {}'.format(epoch, loss))

Epoch: 0, Loss: 1.101180911064148
Epoch: 1, Loss: 1.0939327478408813
Epoch: 2, Loss: 1.0890913009643555
Epoch: 3, Loss: 1.0828611850738525
Epoch: 4, Loss: 1.0769152641296387
Epoch: 5, Loss: 1.069310188293457
Epoch: 6, Loss: 1.062929391860962
Epoch: 7, Loss: 1.0542429685592651
Epoch: 8, Loss: 1.0432710647583008
Epoch: 9, Loss: 1.034690260887146
Epoch: 10, Loss: 1.0252346992492676
Epoch: 11, Loss: 1.0135574340820312
Epoch: 12, Loss: 1.0051668882369995
Epoch: 13, Loss: 0.9967138171195984
Epoch: 14, Loss: 0.9861629009246826
Epoch: 15, Loss: 0.9747775793075562
Epoch: 16, Loss: 0.9632573127746582
Epoch: 17, Loss: 0.9522550106048584
Epoch: 18, Loss: 0.9422412514686584
Epoch: 19, Loss: 0.9296534061431885
Epoch: 20, Loss: 0.91802978515625
Epoch: 21, Loss: 0.9039518237113953
Epoch: 22, Loss: 0.8943111300468445
Epoch: 23, Loss: 0.8813112378120422
Epoch: 24, Loss: 0.8754137754440308
Epoch: 25, Loss: 0.8605828881263733
Epoch: 26, Loss: 0.8511001467704773
Epoch: 27, Loss: 0.8382753729820251
Epoch: 2

In [72]:
model.eval()
_, pred = model(data.x.to(device), data.edge_index.to(device)).max(dim=1)
test_preds = pred[data.test_mask].detach().cpu().numpy()
test_ids = df_nodes['id'][data.test_mask].values

In [73]:
test_preds = labels.inverse_transform(test_preds)

In [74]:
submission_df = pd.DataFrame({'id': test_ids, 'branch': test_preds})
submission_df.to_csv('submission.csv', index=False)
len(submission_df)

3944

In [ ]:
TRAINING WITH DEEPWALK EMBEDDINGS

In [75]:
class DeepWalk:
    def __init__(self, window_size: int, embedding_size: int, walk_length: int, walks_per_node: int):
        """
        :param window_size: window size for the Word2Vec model
        :param embedding_size: size of the final embedding
        :param walk_length: length of the walk
        :param walks_per_node: number of walks per node
        """
        self.window_size = window_size
        self.embedding_size = embedding_size
        self.walk_length = walk_length
        self.walk_per_node = walks_per_node

    def random_walk(self, g: nx.Graph, start: str, use_probabilities: bool = False) -> List[str]:
        """
        Generate a random walk starting on start
        :param g: Graph
        :param start: starting node for the random walk
        :param use_probabilities: if True take into account the weights assigned to each edge to select the next candidate
        :return:
        """
        walk = [start]
        for i in range(self.walk_length):
            neighbours = g.neighbors(walk[i])
            neighs = list(neighbours)
            if use_probabilities:
                probabilities = [g.get_edge_data(walk[i], neig)["weight"] for neig in neighs]
                sum_probabilities = sum(probabilities)
                probabilities = list(map(lambda t: t / sum_probabilities, probabilities))
                p = np.random.choice(neighs, p=probabilities)
            else:
                p = random.choice(neighs)
            walk.append(p)
        return walk

    def get_walks(self, g: nx.Graph, use_probabilities: bool = False) -> List[List[str]]:
        """
        Generate all the random walks
        :param g: Graph
        :param use_probabilities:
        :return:
        """
        random_walks = []
        for _ in range(self.walk_per_node):
            random_nodes = list(g.nodes)
            random.shuffle(random_nodes)
            if len(random_nodes) > 0:
              for node in tqdm(random_nodes):
                  random_walks.append(self.random_walk(g=g, start=node, use_probabilities=use_probabilities))
        return random_walks

    def compute_embeddings(self, walks: List[List[str]]):
        """
        Compute the node embeddings for the generated walks
        :param walks: List of walks
        :return:
        """
        model = Word2Vec(sentences=walks, window=self.window_size, vector_size=self.embedding_size)
        return model.wv

# Load the Zachary's Karate Club graph
g = nx.karate_club_graph()

# Initialize the DeepWalk model
dw = DeepWalk(window_size=5, embedding_size=32, walk_length=10, walks_per_node=80)

# Generate the random walks
walks = dw.get_walks(g, use_probabilities=False)

# Compute the embeddings
embeddings = dw.compute_embeddings(walks)

# Retrieve the embedding of node 1
embeddings = embeddings.vectors

NameError: name 'List' is not defined